# exploration of the data 

in this notebook we'll take a look at the gathered data from the MIMIC-III sql requests as well as the eICU sql requests. 

This data is gathered to be able to train a ML model to be able to predict AKI

In [ ]:
# imports
import plotly.express as px
import pandas as pd
import numpy as np
import os
from pathlib import Path
import re
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
mimic_dir = Path(Path.cwd() / 'output' / 'mimiciii')
eicu_dir = Path(Path.cwd() / 'output' / 'eicu')
mimic_dir

In [ ]:
# list contents and add them in a dict for easier processing:
def get_files(database_dir):
    files_dirs = [x for x in database_dir.glob('**/*.parquet') if x.is_file()]
    files_names = [x.stem for x in files_dirs]
    files = dict(zip(files_names, files_dirs))
    return files
files_mimic = get_files(mimic_dir)
files_eicu = get_files(eicu_dir)
files_mimic

In [ ]:
files_eicu

## Dataframes

We'll import the different dataset in pandas dataframes, to be able to visualise them easily:

In [ ]:
def create_dataframe_dict(parquetdict: dict):
    df_dict = dict()
    df_column_names = dict()
    for k,v in parquetdict.items():
        df_dict[k]=pd.read_parquet(v)
        df_column_names[k] = df_dict[k].columns
    return df_dict,df_column_names
mimiciii_df_dict, mimic_column_names = create_dataframe_dict(files_mimic)
eicu_df_dict, eicu_column_names = create_dataframe_dict(files_eicu)

In [ ]:
mimiciii_df_dict['chart_vitals_stay']

In [ ]:
eicu_df_dict['chart_vitals_stay']

In [ ]:
mimic_column_names

## visualize parameters

In [ ]:
i="chart_vitals_stay"
j="diasbp_mean"
amount_of_data_mimic = mimiciii_df_dict[i][j].count()
amount_of_data_eicu = eicu_df_dict[i][j].count()
fig = make_subplots(rows=1, cols=2, subplot_titles=("mimic data amount: {}".format(amount_of_data_mimic),"eicu data, amount: {}".format(amount_of_data_eicu)))
fig.add_trace(go.Box(y=mimiciii_df_dict[i][j], name='mimic',boxpoints=False,), row=1, col=1)
fig.add_trace(go.Box(y=eicu_df_dict[i][j], name='eicu',boxpoints=False,), row=1, col=2)
fig.update_layout(
    yaxis_title=j,
    boxmode='group' # group together boxes of the different traces for each value of x
)
fig.show()

In [ ]:
def create_plots(df_mimic, df_eicu):
    x = 'icustay_id'
    columns_to_skip = ['hadm_id','icustay_id','charttime','subject_id']
    root_dir = Path(Path.cwd() / 'output'/'data_comparison')

    for i in mimic_column_names:
        for j in mimic_column_names[i]:
            print(i,j)
            if j not in (columns_to_skip):
                if(i=='comorbidities'):
                    x='hadm_id'
                else:
                    x='icustay_id'
                amount_of_data_mimic = df_mimic[i][j].count()
                amount_of_data_eicu = df_eicu[i][j].count()
                fig = make_subplots(rows=1, cols=2, subplot_titles=("mimic data amount: {}".format(amount_of_data_mimic),"eicu data, amount: {}".format(amount_of_data_eicu)))
                fig.add_trace(go.Box(y=df_mimic[i][j], name='mimic',boxpoints=False,), row=1, col=1)
                fig.add_trace(go.Box(y=df_eicu[i][j], name='eicu',boxpoints=False,), row=1, col=2)
                fig.update_layout(
                    yaxis_title=j,
                    boxmode='group' # group together boxes of the different traces for each value of x
                )
                # fig.write_html(root_dir / str(i+"_"+j+".html"))
                fig.write_image(root_dir / str(i+"_"+j+".jpeg"))
create_plots(mimiciii_df_dict, eicu_df_dict)